In [1]:
import carla
import random

import pygame
from pygame.locals import K_ESCAPE
from pygame.locals import K_SPACE
from pygame.locals import K_a, K_d, K_s, K_w, K_c
import numpy as np
import json

VIEW_WIDTH = 1920//2
VIEW_HEIGHT = 1080//2
VIEW_FOV = 120

BB_COLOR = (248, 64, 24)

pygame 2.5.2 (SDL 2.28.3, Python 3.7.12)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
client = carla.Client('127.0.0.1', 2000)
client.set_timeout(2.0)
world = client.get_world()
camera_transform = carla.Transform(carla.Location(x=-5.5, z=2.8), carla.Rotation(pitch=-15))
car_transform = random.choice(world.get_map().get_spawn_points())

display = pygame.display.set_mode((VIEW_WIDTH, VIEW_HEIGHT), pygame.HWSURFACE | pygame.DOUBLEBUF)
image = None
depth_image = None
capture = True

In [3]:
settings = world.get_settings()
settings.synchronous_mode = True
world.apply_settings(settings)

926

In [4]:
car_bp = world.get_blueprint_library().filter('vehicle.*')[0]
car = world.spawn_actor(car_bp, car_transform)

In [5]:
camera_bp = world.get_blueprint_library().find('sensor.camera.rgb')
camera_bp.set_attribute('image_size_x', str(VIEW_WIDTH))
camera_bp.set_attribute('image_size_y', str(VIEW_HEIGHT))
camera_bp.set_attribute('fov', str(VIEW_FOV))

camera = world.spawn_actor(camera_bp, camera_transform, attach_to=car)

In [6]:
depth_camera_bp = world.get_blueprint_library().find('sensor.camera.depth')
depth_camera_bp.set_attribute('image_size_x', str(VIEW_WIDTH))
depth_camera_bp.set_attribute('image_size_y', str(VIEW_HEIGHT))
depth_camera_bp.set_attribute('fov', str(VIEW_FOV))

depth_camera = world.spawn_actor(depth_camera_bp, camera_transform, attach_to=car)

In [7]:
def set_image(img):
    global capture
    if capture:
        global image
        image = img
        capture = False

In [8]:
def set_depth_image(img):
    global depth_image
    depth_image = img

In [9]:
camera.listen(lambda image: set_image(image))
depth_camera.listen(lambda depth_image: set_depth_image(depth_image))

In [10]:
def main():
    pygame.init()
    
    pygame_clock = pygame.time.Clock()
    
    json_ = json.load(open("depth_images/no.json", "r"))
    running = True
    while running:
        world.tick()
        global capture
        capture = True
        pygame_clock.tick_busy_loop(20)

        if image is not None:
            array = np.frombuffer(image.raw_data, dtype=np.dtype("uint8"))
            array = np.reshape(array, (image.height, image.width, 4))
            array = array[:, :, :3]
            array = array[:, :, ::-1]
            surface = pygame.surfarray.make_surface(array.swapaxes(0, 1))
            display.blit(surface, (0, 0))

        keys = pygame.key.get_pressed()
        if keys[K_ESCAPE]:
            running = True
            open("depth_images/no.json", "w").write(json.dumps(json_))
            pygame.quit()

        control = car.get_control()
        control.throttle = 0
        if keys[K_w]:
            control.throttle = 1
            control.reverse = False
        elif keys[K_s]:
            control.throttle = 1
            control.reverse = True
        if keys[K_a]:
            control.steer = max(-1., min(control.steer - 0.05, 0))
        elif keys[K_d]:
            control.steer = min(1., max(control.steer + 0.05, 0))
        elif keys[K_c]:
            if depth_image is not None:
                depth_image.save_to_disk("depth_images/"+str(json_)+".jpg", carla.ColorConverter.LogarithmicDepth)
                json_ += 1
        else:
            control.steer = 0
        control.hand_brake = keys[K_SPACE]

        car.apply_control(control)

        pygame.display.flip()
        pygame.event.pump()

In [11]:
if __name__ == '__main__':
    main()

error: video system not initialized